In [ ]:
import pathlib
import sys
import os
import typing

In [ ]:
sys.path.append(str(pathlib.Path.cwd().parent.parent))
os.chdir(str(pathlib.Path.cwd().parent.parent))
%load_ext autoreload
%autoreload 2

In [ ]:
import random

from riseuplib.model import Eva, Parcel
from riseuplib.pipeline import Pipeline
from riseuplib.runnables import PolygonRunnable, AssociationRunnable, SetbackRunnable, RunnableFactory
from riseuplib.utils.tools import load_config
from riseuplib.utils.helpers import DataProxy
from riseuplib.utils.visuals import create_canvas
from riseuplib.settings import Settings

from pydantic import BaseModel
from pathlib import Path
from shapely import wkt

## Explore Building Model

In [ ]:
eva = Eva("./configs/building_cfg_single.json")
eva.set_position(5, 0, 0)
eva.display(shadow_time=10)

## Create Urban Planning pipeline

In [ ]:
class Config(BaseModel):
    data_paths: str
    runnables: typing.List
        
config = Config(**load_config('./configs/planner_thesis.json'))

In [ ]:
settings = Settings()
runnable_kwargs = {
    "settings": settings
}

factory = RunnableFactory()
factory.register("polygon_runnable", PolygonRunnable)
factory.register("association_runnable", AssociationRunnable)
factory.register("setback_runnable", SetbackRunnable)
pipeline = Pipeline([factory.create(args, **runnable_kwargs) for args in config.runnables])

## Run Urban Planning pipeline

In [ ]:
data_proxy = DataProxy(settings)

In [ ]:
# find by addr
address = "540 Howes Creek Rd, Mansfield VIC 3722"
property_id = data_proxy.find_id_by_address(address)

In [ ]:
# find by loc
point_data = [145.7102847125895, -38.641301523261376]
property_id = data_proxy.find_id_by_location(point_data)

In [ ]:
# if prop id is known
property_id = '0000024b-4e1b-5bd5-b0d2-f77be37b5b65'

In [ ]:
pipeline.run(data={"property_id": property_id},
             zone_permit_exists=True,
             subdivision_permit_exists=True,
             tolerance=0.005)

In [ ]:
pipeline.state

# Single Building

In [ ]:
parcels_data = pipeline.state['parcels_data']

In [ ]:
parcel = parcels_data[0].parcel

In [ ]:
RESULTS_DIR='./results/complex_case_2'

In [ ]:
import pygad
import random
import numpy as np
import matplotlib.pyplot as plt

from riseuplib.model.land import Land
from riseuplib.utils.geometry.basic import discretize
from shapely.geometry import Polygon, Point
from timeit import default_timer as timer


def cost(parcel: Parcel, eva: Eva, minimise=True) -> float:
    result = 0
    x, y, _ = eva.get_position()
    dist_to_front = parcel.setback_shape.sides.front.distance(Point(x, y))
    land_shape = parcel.setback_shape.polygon_utm
    eva_shape = eva.get_shape()
    shadows = eva.cast_shadows()
    
    shadow_component = sum([np.abs(shadow.intersection(parcel.left_parcels[0].polygon_utm).area + 
                                   shadow.intersection(parcel.right_parcels[0].polygon_utm).area) ** 2 for shadow in shadows.values()])
    
    result += dist_to_front ** 2 + np.abs(eva_shape.intersection(land_shape).area - eva_shape.area) ** 2 + eva_shape.distance(
        land_shape) ** 2 + shadow_component

    return result if minimise else -result


class GAAlgorithm:
    """
    Genetic algorithm class instance
    """
    def __init__(self):
        pass

    def run(self,
            parcel: Parcel,
            eva: Eva,
            optimisation_area: Polygon = None,
            init_angle: float = None,
            seed=123,
            num_generations=1000,
            sol_per_pop=5,
            num_parents_mating=3,
            reach=0.0,
            saturation=20,
            *args, **kwargs) -> pygad.GA:

        # set seed (this allows getting reproducible results)
        random.seed(seed)
        np.random.seed(seed)

        visualise = kwargs["visualise"] if "visualise" in kwargs else False
        n_actions = 3

        def fitness_function(solution: typing.List, solution_idx: int) -> float:
            if len(solution) != 3:
                raise RuntimeError(f"Action state dimension is wrong. "
                                   f"Actions len: {len(solution)}")
            x, y, angle = solution
            eva.set_position(x, y, angle)
            return cost(parcel=parcel, eva=eva, minimise=False)

        points = discretize(optimisation_area) if optimisation_area else discretize(parcel.setback_shape.polygon_utm)

        x_space, y_space = list(zip(*points))

        angle_gene_space = list(range(0, 360, 10)) if init_angle is None else list(range(int(init_angle) - 10,
                                                                                   int(init_angle) + 10, 1))
        gene_space = [list(x_space), list(y_space), angle_gene_space]

        # create an instance of genetic algorithm
        ga_instance = pygad.GA(num_generations=num_generations,
                               sol_per_pop=sol_per_pop,
                               num_parents_mating=num_parents_mating,
                               num_genes=n_actions,
                               mutation_type="adaptive",
                               mutation_num_genes=(3, 2),
                               gene_space=gene_space,
                               allow_duplicate_genes=False,
                               fitness_func=fitness_function)

        print("Initial Population")
        print(ga_instance.initial_population)

        # apply first population
        fitness_function(ga_instance.population[0], 0)
        if visualise:
            self.visualise(parcel=parcel, eva=eva)

        # run GA
        start = timer()
        ga_instance.run()
        elapsed = timer() - start
        print(f"Elapsed: {elapsed}")

        # output final population
        print("Final Population")
        print(ga_instance.population)
        print(f"Fitness function value: {fitness_function(ga_instance.best_solution()[0], 0)}")

        if visualise:
            # convergence plot
            ga_instance.plot_fitness(save_dir=os.path.join(RESULTS_DIR, 'fitness.png'))

            self.visualise(parcel=parcel,
                           eva=eva,
                           op_area=optimisation_area)

        return ga_instance

    @staticmethod
    def visualise(parcel, eva, op_area=None, solutions=None):
        plt.figure(figsize=(10, 5))
        plt.xticks(fontsize=18)
        plt.yticks(fontsize=18)
        plt.gca().set_aspect('equal', adjustable='box')
        ax = plt.gca()
        eva.add_view(ax)
        land_shape = parcel.setback_shape.polygon_utm
        ax.plot(*land_shape.exterior.xy, "g")
        if op_area is not None:
            ax.fill(*op_area.exterior.xy, "b", alpha=0.2)
        if solutions is not None:
            for s in solutions:
                ax.plot(s[0], s[1], "*r")
        ax.grid()
        plt.show()

In [ ]:
p = parcel.setback_shape.polygon_utm.representative_point()
eva.set_position(p.x, p.y, 0)
ga = GAAlgorithm()
result = ga.run(parcel=parcel,
       eva=eva,
       visualise=True,
       num_generations=2000, 
       seed=random.randint(0, 10000))
solution = tuple(result.best_solution()[0])

In [ ]:
land = Land(parcel=parcel)
eva.default_position = solution
land.place_eva(eva)
land.display(setback_poly=parcel.setback_shape.polygon_utm, 
             save_data=True, 
             save_path=os.path.join(RESULTS_DIR, 'final_result.png'))

In [ ]:
plt.figure(figsize=(10, 5))
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.gca().set_aspect('equal', adjustable='box')
ax = plt.gca()
shadows = eva.cast_shadows()
ax.fill(*parcel.polygon_utm.exterior.xy, facecolor="forestgreen", edgecolor="black", alpha=0.4)

eva.add_view(ax)

for p in parcel.left_parcels:
    ax.plot(*p.polygon_utm.exterior.xy, 'r', label='Left Parcel')
for p in parcel.right_parcels:
    ax.plot(*p.polygon_utm.exterior.xy, 'b', label='Right Parcel')

for t, s in shadows.items():
    ax.fill(*s.exterior.xy, 'k', alpha=0.2, label=f'Shadows')

plt.axis('off')
handles, labels = ax.get_legend_handles_labels()
unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
plt.legend(*zip(*unique), loc='upper right')
plt.savefig(os.path.join(RESULTS_DIR, 'shadows.png'), dpi=300, bbox_inches='tight', pad_inches=0)

## Multiple Buildings

In [ ]:
property_id = '0e9ec792-e7e0-53ea-8137-d03b045a6347' # 9 properties

In [ ]:
pipeline.run(data={"property_id": property_id},
             zone_permit_exists=True,
             subdivision_permit_exists=True,
             tolerance=0.005)

In [ ]:
parcels = [p.parcel for p in parcels_data]
evas = [Eva("./configs/building_cfg_single.json") for _ in range(len(parcels))]

In [ ]:
RESULTS_DIR='./results/complex_case_2'

In [ ]:
import pygad
import random
import numpy as np
import matplotlib.pyplot as plt

from riseuplib.model.land import Land
from riseuplib.utils.geometry.basic import discretize
from shapely.geometry import Polygon, Point
from timeit import default_timer as timer


def cost_n(parcels: typing.List[Parcel], evas: typing.List[Eva], minimise=True) -> float:
    result = 0
    for eva, parcel in zip(evas, parcels):
        x, y, _ = eva.get_position()
        dist_to_front = parcel.setback_shape.sides.front.distance(Point(x, y))
        land_shape = parcel.setback_shape.polygon_utm
        eva_shape = eva.get_shape()
        result += 10 * dist_to_front ** 2 + 10 * np.abs(eva_shape.intersection(land_shape).area - eva_shape.area) ** 2 + eva_shape.distance(land_shape) ** 2

    return result if minimise else -result


class GAAlgorithm:
    """
    Genetic algorithm class instance
    """
    def __init__(self):
        pass

    def run(self,
            parcels: typing.List[Parcel],
            evas: typing.List[Eva],
            init_angle: float = None,
            seed=123,
            num_generations=1000,
            sol_per_pop=5,
            num_parents_mating=3,
            reach=0.0,
            saturation=20,
            *args, **kwargs) -> pygad.GA:

        # set seed (this allows getting reproducible results)
        random.seed(seed)
        np.random.seed(seed)

        visualise = kwargs["visualise"] if "visualise" in kwargs else False
        n_actions = 3 * len(parcels)

        def fitness_function(solution: typing.List, solution_idx: int) -> float:
            if len(solution) != 3 * len(evas):
                raise RuntimeError(f"Action state dimension is wrong. "
                                   f"Actions len: {len(solution)}")
            for idx, eva in enumerate(evas):
                x, y, angle = solution[idx * 3: idx * 3 + 3]
                eva.set_position(x, y, angle)
            return cost_n(parcels=parcels, evas=evas, minimise=False)
        
        gene_space = []
        for parcel in parcels:
            points = discretize(parcel.setback_shape.polygon_utm)
            x_space, y_space = list(zip(*points))
            angle_gene_space = list(range(0, 360, 10)) if init_angle is None else list(range(int(init_angle) - 10,
                                                                                       int(init_angle) + 10, 1))
            gene_space.extend([list(x_space), list(y_space), angle_gene_space])
        

        # create an instance of genetic algorithm
        ga_instance = pygad.GA(num_generations=num_generations,
                               sol_per_pop=sol_per_pop,
                               num_parents_mating=num_parents_mating,
                               num_genes=n_actions,
                               mutation_type="adaptive",
                               mutation_num_genes=(3, 2),
                               gene_space=gene_space,
                               allow_duplicate_genes=False,
                               fitness_func=fitness_function)

        print("Initial Population")
        print(ga_instance.initial_population)

        # apply first population
        fitness_function(ga_instance.population[0], 0)
        if visualise:
            self.visualise(parcels=parcels, evas=evas)

        # run GA over all buildings on land
        start = timer()
        ga_instance.run()
        elapsed = timer() - start
        print(f"Elapsed: {elapsed}")

        # output final population
        print("Final Population")
        print(ga_instance.population)
        print(f"Fitness function value: {fitness_function(ga_instance.best_solution()[0], 0)}")

        if visualise:
            # convergence plot
            ga_instance.plot_fitness(save_dir=os.path.join(RESULTS_DIR, 'fitness.png'))

            self.visualise(parcels=parcels,
                           evas=evas)

        return ga_instance

    @staticmethod
    def visualise(parcels, evas, op_area=None, solutions=None):
        plt.figure(figsize=(10, 5))
        plt.xticks(fontsize=18)
        plt.yticks(fontsize=18)
        plt.gca().set_aspect('equal', adjustable='box')
        ax = plt.gca()
        for eva, parcel in zip(evas, parcels):
            eva.add_view(ax)
            land_shape = parcel.setback_shape.polygon_utm
            ax.plot(*land_shape.exterior.xy, "g")
            if op_area is not None:
                ax.fill(*op_area.exterior.xy, "b", alpha=0.2)
            if solutions is not None:
                for s in solutions:
                    ax.plot(s[0], s[1], "*r")
        ax.grid()
        plt.show()

In [ ]:
for eva, parcel in zip(evas, parcels):
    p = parcel.setback_shape.polygon_utm.representative_point()
    eva.set_position(p.x, p.y, 0)

In [ ]:
GAAlgorithm.visualise(parcels=parcels, evas=evas)

In [ ]:
ga = GAAlgorithm()
result = ga.run(parcels=parcels,
                evas=evas,
                visualise=True,
                num_generations=10000, 
                seed=random.randint(0, 10000))

solution = tuple(result.best_solution()[0])

In [ ]:
save_path=os.path.join(RESULTS_DIR, 'final_result.png')

In [ ]:
lands = []
for idx, (eva, parcel) in enumerate(zip(evas, parcels)):
    land = Land(parcel=parcel)
    eva.default_position = solution[idx*3:idx*3 + 3]
    land.place_eva(eva)
    lands.append(land)

In [ ]:
min_x = min_y = float('inf')
max_x = max_y = -float('inf')
for land in lands:
    minx, maxx = land.get_xrange(offset=10)
    miny, maxy = land.get_yrange(offset=10)
    min_x = min(min_x, minx)
    min_y = min(min_y, miny)
    max_x = max(max_x, maxx)
    max_y = max(max_y, maxy)

ax = create_canvas((10, 10), min_x, min_y, max_x, max_y)

# visualise land itself
for land in lands:
    land.add_view(ax=ax, facecolor="forestgreen", edgecolor="black", alpha=0.4)

# add hatched setback polygon
for parcel in parcels:
    ax.fill(*parcel.setback_shape.polygon_utm.exterior.xy, fc="none", ec="k", hatch="/", label="Buildable area")

# visualise buildings
for eva in evas:
    eva.add_view(ax, show_origin=False)

# reduce the amount of white space in the figure
plt.tight_layout()
# plt.show()
plt.savefig(save_path, dpi=300, bbox_inches='tight', pad_inches=0)